In [22]:
import arviz as az
import networkx as nx
import numpy as np
import pandas as pd
from pywhy_graphs.viz import draw
import pymc as pm
from scipy import stats
import warnings

In [23]:
%config InlineBackend.figure_format = 'retina'
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")
warnings.filterwarnings('ignore')

The data in data(Monks) are “like” and “dislike” nominations by 18 monks living in the same monastery over three time periods. Therefore the observed variables are counts from 0 to 3 of times monk A nominated monk B as liked or disliked.1 Each row in the data is a pair of monks (a dyad). The variables are:
* A: Index number for first monk in dyad
* B: Index number for second monk in dyad
* like_AB: Number of times A nominated B as liked
* like_BA: Number of times B nominated A as liked
* dislike_AB: Number of times A nominated B as disliked • dislike_BA: Number of times B nominated A as disliked

1\. Use these data to estimate the amount of reciprocity in “like” nominations within dyads. You can ignore the “dislike” data for now. Use the social network example from the book to help, but you should modify it appropriately.


In [14]:
d = pd.read_csv("../../Data/week08_Monks.csv", sep=",", header=0)
d

,dyad_id,A,B,like_AB,like_BA,dislike_AB,dislike_BA,A_name,B_name
0,1,1,2,0,3,0,0,ROMUL,BONAVEN
1,2,1,3,3,3,0,0,ROMUL,AMBROSE
2,3,1,4,0,0,0,0,ROMUL,BERTH
3,4,1,5,0,0,0,0,ROMUL,PETER
4,5,1,6,0,0,0,0,ROMUL,LOUIS
...,...,...,...,...,...,...,...,...,...
148,149,15,17,0,0,0,0,AMAND,ELIAS
149,150,15,18,0,0,0,0,AMAND,SIMP
150,151,16,17,0,0,0,0,BASIL,ELIAS
151,152,16,18,0,0,0,0,BASIL,SIMP


In [15]:
d.dtypes

dyad_id        int64
A              int64
B              int64
like_AB        int64
like_BA        int64
dislike_AB     int64
dislike_BA     int64
A_name        object
B_name        object
dtype: object

In [21]:
like_AB = d.like_AB
like_BA = d.like_BA

with pm.Model() as m1:
    sigma_T = pm.Exponential.dist(1)
    rho, _, _ = pm.LKJCholeskyCov("rho", eta=4, n=2, sd_dist=sigma)
    a_bar = pm.Normal("a_bar", 0., 1., shape=2)
    v = pm.MvNormal("v", a_bar, chol=rho, shape=(num_Ds, 2))

    a = pm.Normal("a", 0., 1.)
    pBA = pm.invlogit(a + T[:, 1])
    pAB = pm.invlogit(a + T[:, 0])
    like_AB = pm.Binomial("like_AB", n=3, p=pAB, observed=like_AB),
    like_BA = pm.Binomial("like_BA", n=3, p=pBA, observed=like_BA),

2\. Now also analyze the “dislike” nominations. Estimate the amount of reciprocity in the “dislike” nominations and compare to the estimate from the “like” nomi- nations. Does “like” or “dislike” have higher reciprocity? Be sure to compute the posterior contrast between the two estimates.

3\. Now consider generalized liking and disliking. Add generalized “receiving” pa- rameters to the model, analogous to receiving varying effects from the sharing ex- ample in the book/lecture. (Each monk only named 3 likes and 3 dislikes each time, so the amount of “giving” is fixed by design in these data.) Can you identify any individuals who are particularly liked/disliked, independent of the dyadic relations?